# Exploring non-nanosecond time implementations on Pandas Data Structures

This notebook demonstrates the current state of the implementation for non-nanosecond datetimes as detailed in [PR 47675](https://github.com/pandas-dev/pandas/pull/47675/files). The pandas version is printed
at the beginning so that you can easily look at the exact commits that are included in the nightly build.

To ensure that this notebook runs properly, please install the environment or refer to [ENVIRONMENT.md](../ENVIRONMENT.md)


Helpful Links:
* [NumPy DateTime Units](https://numpy.org/doc/stable/reference/arrays.datetime.html#datetime-units)
* [Original Proposal](https://numpy.org/doc/1.13/neps/datetime-proposal.html)

In [1]:
import pathlib
import logging
import sys

import numpy as np
import pandas as pd

from IPython.display import display, Markdown

# Setup Logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)

logger.addHandler(handler)

display(Markdown(f'''
|Package| Version         |
|:------|----------------:|
|Python | {sys.version}   |
|Pandas | {pd.__version__}|
|Numpy  | {np.__version__}|
'''))


|Package| Version         |
|:------|----------------:|
|Python | 3.10.8 (main, Nov  4 2022, 08:45:25) [Clang 12.0.0 ]   |
|Pandas | 2.0.0.dev0+724.ga6d3e13b3a|
|Numpy  | 1.23.4|


### pandas `date_range`

We recast the resolution on the date_range object. However, creating timestamps older than 1700-01-01 results in out of bounds

In [2]:
dr = pd.date_range('1700', periods=100).astype('datetime64[ns]')
dr[:5]

DatetimeIndex(['1700-01-01', '1700-01-02', '1700-01-03', '1700-01-04',
               '1700-01-05'],
              dtype='datetime64[ns]', freq='D')

In [3]:
dr_seconds = dr.astype('datetime64[s]')
dr_seconds[:5]

DatetimeIndex(['1700-01-01', '1700-01-02', '1700-01-03', '1700-01-04',
               '1700-01-05'],
              dtype='datetime64[s]', freq=None)

In [4]:
try:
    dr_older = pd.date_range('1500', periods=100).astype('datetime64[ns]')
except:
    logger.error('Pandas OutOfBoundsDateTime Exception Raised')

Pandas OutOfBoundsDateTime Exception Raised


### pandas `Series`

See above; same result

In [5]:
seconds = pd.Series(['1700-01-01'], dtype='datetime64[s]')
nanoseconds = pd.Series(['1700-01-01'], dtype='datetime64[ns]')

seconds, nanoseconds

(0   1700-01-01
 dtype: datetime64[s],
 0   1700-01-01
 dtype: datetime64[ns])

In [6]:
try:
    seconds = pd.Series(['1500-01-01'], dtype='datetime64[s]')
except:
    logger.error('Pandas DateTime OutOfBounds Exception Raised')

Pandas DateTime OutOfBounds Exception Raised


## pandas powered by NumPy

The problems that we ran into above can be surpressed by using numpy as the input array. We will explore a set of examples in this section:

1) A pandas.Series whose dates ranges exist outside of the currently known limits for 9 digit precision (1700-01-01 to 2262-04-11)
2) A pandas.DataFrame whose index is the aforementioned series with 2 columns:
    * FARENHEIGHT - Randomly generated floating point
    * KELVIN - Randomly generated float

We will then explore doing the same things millions of years in the past and in the future. 


In [7]:
start_year = 1300
stop_year = 2300

arr = np.arange(np.datetime64(f"{start_year}-01-01"), np.datetime64(f"{stop_year}-01-01"), np.timedelta64(10, "Y"), dtype='datetime64[Y]').astype("M8[s]")
series = pd.Series(arr)

The above line generates a set of datetimes outside the accepted range since we are using second precision. This is an example that works as such:
* Generate dates using arange 
* Inside the arange, use datetime as Years to skip ns time non-linearity 
* Cast the units to seconds using astype
* Generate a series from the resulting array

Note that in this case, that pandas is able to represnt dates well out of its range.

In [8]:
series.resample('50Y')

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'RangeIndex'

Basic resampling of the periods do not work on pandas series objects. What if we set it to an index on a dataframe?

In [9]:
temp_f = np.random.random_sample((len(series),))*100-10 # Keep in Farenheight to demo negative numbers 
temp_k = temp_f * (5/9)+275.15
df = pd.DataFrame({'f': temp_f, 'k': temp_k}, index=series)

The following example looks at slicing by index in pandas. Note the date - we are using non-ns timeseries.

In [10]:
df[df.index < np.datetime64('1400-01-01')]

,f,k
1300-01-01,9.519415,280.438564
1310-01-01,-8.359774,270.505681
1320-01-01,31.131706,292.445392
1330-01-01,80.627611,319.943117
1340-01-01,23.277231,288.081795
1350-01-01,-1.630995,274.243892
1360-01-01,55.523783,305.996546
1370-01-01,52.438165,304.282314
1380-01-01,26.172441,289.690245
1390-01-01,-5.031728,272.354596


In [11]:
df[df.index < np.datetime64('1400-01-01')]['f'].min()

-8.359774097104447